Click the button below if you want to see the code behind the widgets. If you do click  and want to go back to the 'no-code' view, scroll down until you find the 'hide code' button.

In [1]:
"""
MIT License

Copyright (c) 2020 Sylvain Barde - University of Kent

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.
"""

import ipywidgets as widgets
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
from matplotlib import cm
from matplotlib.colors import ListedColormap


import numpy as np
from matplotlib import rcParams
from IPython.display import display, HTML

javascript_functions = {False: "hide()", True: "show()"}
button_descriptions  = {False: "Click to show code", True: "Click to hide code"}

def toggle_code(state):
    """
    Toggles the JavaScript show()/hide() function on the div.input element.
    """

    output_string = "<script>$(\"div.input\").{}</script>"
    output_args   = (javascript_functions[state],)
    output        = output_string.format(*output_args)

    display(HTML(output))
    
def button_action(value):
    """
    Calls the toggle_code function and updates the button description.
    """
    state = value.new

    toggle_code(state)

    value.owner.description = button_descriptions[state]


def utility_widget(curve_init = 'CD', a_init = 0.5, xMin_init = 0, xMax_init = 12, xPrice_init = 16,
                   yPrice_init = 10, budget_init = 100,
                      elev_init = 30, azim_init = 290, zCut_init = False,
                  constraintCut_init = False):
    
    # Declare widgets for interactive input
    curve_list = widgets.Dropdown(options=['CD'],
                                value = curve_init,
                                description='function:',
                                disabled=False)
    a_slider = widgets.FloatSlider(min=0,
                                 max=1,
                                 step=0.1,
                                 description=r'$\alpha$:',
                                 value = a_init,
                                 continuous_update =False)
    xMin_slider = widgets.IntSlider(min=-100,
                                 max=0,
                                 step=1,
                                 description=r'Min. $x$,$y$:',
                                 value = xMin_init,
                                 continuous_update =False)    
    xMax_slider = widgets.IntSlider(min=0,
                                 max=100,
                                 step=1,
                                 description=r'Max. $x$,$y$:',
                                 value = xMax_init,
                                 continuous_update =False)
    xPrice_slider = widgets.FloatSlider(min=0,
                                 max=100,
                                 step=1,
                                 description=r'Price of $x$:',
                                 value = xPrice_init,
                                 continuous_update =False)
    yPrice_slider = widgets.FloatSlider(min=0,
                                 max=100,
                                 step=1,
                                 description=r'Price of $y$:',
                                 value = yPrice_init,
                                 continuous_update =False)
    budget_slider = widgets.FloatSlider(min=0,
                                 max=500,
                                 step=1,
                                 description=r'Budget $M$:',
                                 value = budget_init,
                                 continuous_update =False)
    elev_slider = widgets.FloatSlider(min=-90,
                                 max=90,
                                 description= 'Elevation:',
                                 value = elev_init,
                                 continuous_update =False)
    azim_slider = widgets.FloatSlider(min=0,
                                 max=360,
                                 description= 'Azimuth:',
                                 value = azim_init,
                                 continuous_update =False)
    zCut_check = widgets.Checkbox(value = zCut_init,
                                   description='Fix $U$',
                                   disabled=False,
                                   indent=True)
    constraintCut_check = widgets.Checkbox(value = constraintCut_init,
                                   description='Fix $M$',
                                   disabled=False,
                                   indent=True)  
    
    # Link widgets as required
    
    def optim_plot(curve, a, xMin, xMax, xPrice, yPrice, budget, elev, azim , zCut, constraintCut):

        fundict = {'CD': lambda x, y: (x**a)*(y**(1-a)),
                  }
        fundictAlt = {'CD': lambda x, z: (x**(-a/(1-a)))*(z**(1/(1-a))),
                  }

        if zCut is True and constraintCut is False:
            zCut = False
        
        f = fundict[curve]
        fAlt = fundictAlt[curve]

        res = 100
        xFull = np.linspace(1e-3, xMax, res)
        yFull = np.linspace(1e-3, xMax, res)
        XFull, YFull = np.meshgrid(xFull, yFull)
        ZFull = f(XFull, YFull)
        zMax = max(ZFull.flatten())

        if constraintCut is True:
            xConstraint = np.linspace(0, budget/xPrice, res)
            yConstraint = budget/yPrice - xPrice/yPrice*xConstraint
            zConstraint = f(xConstraint, yConstraint)
            optInd = np.argmax(zConstraint)
            xOpt = xConstraint[optInd]
            yOpt = yConstraint[optInd]
            zVal = zConstraint[optInd]
    
        if zCut is True:
            resZ = int(np.floor((zVal/zMax)*res))
            z = np.linspace(0, zVal, resZ)
            zCover = (zVal/zMax)
            viridis = cm.get_cmap('viridis', 512)
            myCmap = ListedColormap(
                       viridis(np.linspace(0, zCover, 256))
                       )
        else:
            z = np.linspace(0, zMax, res)
            myCmap = 'viridis'

        X, Z = np.meshgrid(xFull, z)
        Y = fAlt(X, Z)
        
        Y = np.clip(Y,0,xMax)
        
        # Slice  budget constraint 
        if constraintCut is True:
            mask = Y < budget/yPrice - xPrice/yPrice*X
            Y[mask] = budget/yPrice - xPrice/yPrice*X[mask]
            
        Z = f(X,Y)
        
        # Create figure
        mrkrSize = 2*rcParams['lines.markersize'] ** 2
        fig = plt.figure(figsize=(20,10))

        # Plot 3D utility function
        ax = fig.add_subplot(1, 2, 1, projection='3d')
        ax.plot_surface(X, Y, Z, cmap=myCmap)
        
        if constraintCut is True:
            ax.plot3D(xConstraint, yConstraint, zConstraint, 'b', linewidth=2)
            ax.plot3D(xConstraint, yConstraint, 0*zConstraint, 'b--', linewidth=2, zorder = -5)
        
        if zCut is True:
            xIC = X[-1,:]
            yIC = Y[-1,:]
            zIC = Z[-1,:]
            maskIC = yIC < xMax
            ax.plot3D(xIC[maskIC], yIC[maskIC], zIC[maskIC], 'r', linewidth=2)
            ax.plot3D(xIC[maskIC], yIC[maskIC], 0*zIC[maskIC], 'r--', linewidth=2, zorder = -5)
        
        if constraintCut is True and zCut is True:
            ax.plot3D([xOpt],[yOpt], [zVal], marker = 'o', color = 'r', zorder = 5)
            ax.plot3D([0,xOpt],[yOpt,yOpt],[0,0],'r--',linewidth=1)
            ax.plot3D([xOpt,xOpt],[0,yOpt],[0,0],'r--',linewidth=1)
            ax.plot3D([xOpt,xOpt],[yOpt,yOpt],[0,zVal],'r--',linewidth=1)


        ax.set_xlim(xMin,xMax)
        ax.set_ylim(xMin,xMax)
        ax.set_zlim(0,1.5*zMax)
        ax.view_init(elev, azim)
        ax.set_xlabel(r'$x$', fontdict = {'fontsize': 25},position=(1, 0))
        ax.set_ylabel(r'$y$', fontdict = {'fontsize': 25},position=(0, 1))
        ax.set_zlabel(r'$U(x,y)$', fontdict = {'fontsize': 25},position=(0, 1), rotation=0)
        
        # Plot Indifference curve map
        ax = fig.add_subplot(1, 2, 2)
        numIC = 6
        zStepVec = np.linspace(zMax/numIC, zMax-1/numIC, numIC)
        for zStep in zStepVec:
            ax.plot(xFull, fAlt(xFull, zStep), 'k--', alpha=0.3)
        
        if constraintCut is True:
            ax.plot(xConstraint, yConstraint, 'b', linewidth=2, alpha=0.6,
                    label=r'Budget constraint')
            
        if zCut is True:
            ax.plot(xFull, fAlt(xFull, zVal), 'r', linewidth=2, alpha=0.6,
                    label=r'Highest reachable IC')
            foc = (xPrice/yPrice)*((1-a)/a)
            ax.plot([0,xMax],[0,foc*xMax],'g--',linewidth=2, alpha=0.6,
                    label=r'FOC $y=\frac{(1-\alpha) p_x}{\alpha p_y} x$')
            # Add markers for the optimal point points, with dotted lines
            ax.scatter(xConstraint[optInd], yConstraint[optInd], s=mrkrSize, c='r', alpha=0.6,
                        label='Optimal point')
            ax.plot([0,xOpt],[yOpt,yOpt],'r--',linewidth=1)
            ax.plot([xOpt,xOpt],[0,yOpt],'r--',linewidth=1)

        if constraintCut is True:
            ax.legend(loc='upper right', frameon=False,prop={'size':20})
        ax.autoscale(enable=True, axis='both', tight=True)
        ax.set_ylim(top = xMax, bottom = 0)
        ax.set_xlim(right = xMax, left = 0)
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
        ax.set_xlabel(r'$x$', fontdict = {'fontsize': 25},position=(1, 0))
        ax.set_ylabel(r'$y$', fontdict = {'fontsize': 25},position=(0, 1), rotation=0)
        ax.plot(1, 0, ">k", transform=ax.get_yaxis_transform(), clip_on=False)
        ax.plot(0, 1, "^k", transform=ax.get_xaxis_transform(), clip_on=False)
        ax.tick_params(labelsize=20)
        
        plt.tight_layout()

    out = widgets.interactive_output(optim_plot, {'curve': curve_list,
                                                   'a': a_slider,
                                                   'xMin': xMin_slider,
                                                   'xMax': xMax_slider,
                                                   'xPrice': xPrice_slider,
                                                   'yPrice': yPrice_slider,
                                                   'budget':budget_slider,
                                                   'elev': elev_slider,
                                                   'azim': azim_slider,
                                                   'zCut': zCut_check,
                                                   'constraintCut': constraintCut_check})
        
    output = widgets.VBox([out,
                  widgets.HBox([curve_list,
                                a_slider, 
                                budget_slider]),
                  widgets.HBox([xMin_slider,
                                xMax_slider,
                                xPrice_slider,
                                yPrice_slider]),
                  widgets.HBox([elev_slider, 
                                azim_slider,
                                constraintCut_check,
                                zCut_check])
                          ])
    display(output)

In [2]:
state = False
toggle_code(state)

button = widgets.ToggleButton(state, description = button_descriptions[state])
button.observe(button_action, "value")

display(button)

ToggleButton(value=False, description='Click to show code')

### The workshorse of economics: isoelastic and Cobb-Douglas functions

For this last notebook, I want to provide a bit of a discussion of the reason you keep seeing a specific type of multivariate function in all the examples and exercises you've been doing over last few weeks.

Many of the multivariate differentiation examples in the previous weeks, especially those with an economics flavour, have used functions like $x^{\frac{1}{3}}y^{\frac{2}{3}}$. Why is this type of function so popular with economists? It all has to do with several desirable mathematical properties that these functions have.

#### Isoelastic functions

While the name **isoelastic** might seem a bit grand and scary, as I said you've actually seen these functions already and worked with them. In particular, they've already been discussed in the notebook for week 5 on why economists love logs. We saw that if we picked a very specific equation for demand, we could end up in a 'best of both worlds' scenario:
- A demand equation that is a non-linear function, which is much more realistic than the 'straight-line' functions often shown in textbook diagrams.
- The equation can nevertheless be turned into a linear one (which is easier to work with) if you take logs.

As we discussed at the time, the **special** equation that gives us this property is the one we've since seen a lot of:

$$z = x^\alpha y^\beta$$

Where $\alpha$ and $\beta$ are two parameters. As explained in previous notebooks, if we take logs of both sides of this equation, we get an equation that is linear in the logarithms of $x$ and $y$:

$$\ln z = \alpha \ln x + \beta \ln y$$

So, you might wonder, why the name ***isoelastic***? Well, it comes from the fact that the elasticities of $z$ with respect to $x$ and with respect to $y$ are **constants**, hence 'iso' (same) elastic. This can be shown pretty easily by taking the partial derivative of $z$ with respect to $x$

$$\frac{\partial z}{\partial x} = \alpha x^{\alpha-1} y^{\beta}$$

We can replace this into the formula for the elasticity of $z$ with respect to $x$ and rearrange:

$$\begin{aligned}
\epsilon_x^z \; & = \; \frac{\partial z}{\partial x}\frac{x}{z}\\
\epsilon_x^z \; & = \; \alpha x^{\alpha-1} y^{\beta}\frac{x}{z}\\
\epsilon_x^z \; & = \; \alpha \frac{x^{\alpha} y^{\beta}}{z}\\
\end{aligned}$$

Given that $z = x^\alpha y^{\beta}$, we can replace this in the numerator. You can see that the $z$ terms then cancel out, leaving $\alpha$ as the result.

$$\epsilon_x^z = \alpha \frac{z}{z} = \alpha$$

So this means that the $\alpha$ and $\beta$ parameters of the isoelastic function $z = x^\alpha y^\beta$ are actually the ***elasticities*** of the function. This is a very useful property, as it tells us that we can easily parameterise the function for a real-life scenario just by providing the elasticities. These can be estimated from data. In Statistics next term, and in econometrics next year you will be taught that if you have data for $x$,$y$,$z$ and you take the logarithm of the variables, you can use linear regression to estimate the log-linear form of the equation and get estimates for elasticities $\alpha$ and $\beta$.

$$\ln z = \alpha \ln x + \beta \ln y$$

It's early day for this, so don;t worry too much about it at this stage, but this is one of the main reasons these kinds of functions are used in economics.

#### The Cobb-Douglas function as a special case

We've already seen the Cobb-Douglas function in last week's workbook, and you've already kind of worked on them in the exercises on multivariate derivatives. This type of function is an isoelastic function with the extra restriction that the elasticities add to one. if we look at the example above, this means assuming that $\beta = 1-\alpha$.

$$z = x^\alpha y^{1-\alpha}$$

An example of where this might be used in economic is a production function, where output is produced with capital $K$ and labour $L$. This is actually an example covered in the weekly exercises.

$$Q(K,L) = K^\alpha L^{1-\alpha}$$

Given that this type of function is isoelastic, it all the benefits discussed above, in terms of be able to interpret $\alpha$ as the elasticity of output with respect to capital and $1-\alpha$ as the elasticity of output with respect to labour. If you increase capital by 1%, output increases by $\alpha$%, and if you increase labour by 1% output increases by $(1-\alpha)$%. Note that the $\beta = 1-\alpha$ restriction therefore means that if you increase both capital and labour by 1%, output also increases by 1%. This is something that we tend to observe empirically, which is why this restriction is often used.




In [3]:
utility_widget()

#### The (second) hidden secret of isoelastic functions: easy constrained optimisation

Isoelastic functions have a very neat trick up their sleeves that makes them particularly useful for constrained optimisation. We'll use a general isoelastic function in the maths below, but this also works for the Cobb-Douglas case where you have $\beta = 1-\alpha$ (this is the case in the interactive widget above).

Take the following constrained optimisation problem, where you have to maximise an isoelastic function, subject to a budget constraint. This could be a production function example, or a utility example like in last week's notebooks.

$$\left\{\begin{aligned}
{\rm Max} \quad & z = x^\alpha y^\beta\\
{\rm s.t.} \quad &  p_x x + p_y y= M\\
\end{aligned}\right.$$

The Lagrangian for this problem can be written out as:

$$\mathcal{L} = x^\alpha y^\beta + \lambda (M - p_x x - p_y y)$$

To solve for the constrained maximum in the original problem, we just have to find the unconstrained maximum of the Lagragian. The first order partial derivatives are:

$$\left\{\begin{aligned}
\frac{\partial \mathcal{L}}{\partial x}  \quad & = \quad \alpha x^{\alpha-1} y^\beta \; - \; \lambda p_x \\
\frac{\partial \mathcal{L}}{\partial y}  \quad & = \quad \beta x^\alpha y^{\beta-1} \; - \; \lambda p_y \\
\frac{\partial \mathcal{L}}{\partial \lambda}  \quad & = \quad M - p_x x - p_y y\\
\end{aligned}\right.$$

To find the maximum, we need to set them equal to zero and solve the resulting system of equations. At first glance, this seems a bit tricky given that the partial derivatives look complicated. In fact, this complication is only **skin deep**, everything actually simplifies down to a neat result. First, let's tidy up the first two partial derivatives.

$$\left\{\begin{aligned}
\alpha x^{\alpha-1} y^\beta \; & = \; \lambda p_x\\
\beta x^\alpha y^{\beta-1} \; & = \; \lambda p_y\\
\end{aligned}\right.$$

Next, we divide the first by the second, side by side. Remember, this gives us the tangency condition I mentioned in the video, where the ratio of the slopes of the function we want to maximise equals the ratio of the slopes of the constraint. As we will see, it is at this point that most of the **apparent** complication actually simplifies away.

$$\frac{\alpha x^{\alpha-1} y^\beta}{\beta x^\alpha y^{\beta-1} } \; = \;  \frac{\lambda p_x}{\lambda p_y}$$

The obvious simplification is that on the right hand side, $\lambda$ disappears. But on the right hand side, the $\alpha$ power on $x$ and the $\beta$ power on $y$ also simplify, as they appear both in the numerator and denominator. What we are left with is:

$$\begin{aligned}
\frac{\alpha x^{-1} }{\beta y^{-1}}  \; & = \;  \frac{p_x}{p_y}\\
\frac{\alpha y }{\beta x} \; & = \;  \frac{p_x}{p_y}\\
\end{aligned}$$

We can rearrange and interpret this in several ways. The first is a statement about the ratio of goods $x$ and $y$ at the optimal solution.

$$\frac{ y }{x} \;  = \;  \frac{p_x }{p_y }\frac{\beta}{\alpha}$$

Given that $\alpha$, $\beta$, $p_x$ and $p_y$ are all **fixed** parameters, this essentially means that the optimal ratio of $x$ and $y$, which maximises the value of $z$, is going to be a **fixed constant**, determined by the prices and elasticities. The higher the value of $\beta$ relative to $\alpha$ and the lower $p_y$ relative to $p_x$, the more of good $y$ you want relative to good $x$. This is particularly useful when using a Cobb-Douglas production function with capital and labour, as this result directly gives us the optimal capital/labour ratio, i.e. the optimal mix of workers and machines.

The other interpretation (as always!) is geometric. If we multiply on both sides of the expression by $x$, we get:

$$y \;  = \;  \frac{p_x }{p_y }\frac{\beta}{\alpha}x$$

This gives us the optimal $y$ as a linear function of $x$, given prices and elasticities. This linear equation, which we built by using the first two FOCs of the Lagrangian, can be combined with the third FOC (the budget constraint) gives us the following system of linear equations:

$$\left\{\begin{aligned}
y \; & = \;  \frac{p_x }{p_y }\frac{\beta}{\alpha}x\\
M \; & = \;d p_x x + p_y y\\
\end{aligned}\right.$$

The first equation is the combinations of $x$ and $y$ that satisfies the tangency condition (and are therefore 'optimal'). In the widget above, this is shows as a green dotted line once you fix the budget $M$ and the utility $U$. The second equation is the combinations of $x$ and $y$ that satisfies the budget constraint, and that is the blue line in the diagram, obtained by fixing $M$. As is standard for systems of linear equations, the solution to the problem is at the intersection of the two straight lines. In the widget above, if you select some prices and an elasticity $\alpha$, you can increase/reduce the value of the budget and you will see that the optimal point simply moves up and down this fixed and linear first order condition.

#### Wrapping up...

The purpose of this final notebook was to show why the isoelastic function $x^\alpha y^\beta$ is so common in economics. The answer, as I hope I've convinced you, is that despite looking a bit complicated at first glance due to the power terms, the function is actually very simple to work with: 
- It's various elasticities are actually given directly by the $\alpha$ and $\beta$ parameters, so not only do you not need to derive them (just the once to convince yourselves!), but they can easily be estimated from data. This will be a core focus of next year's econometrics modules.
- Finding the constrained maximum of such a function essentially boils down to solving a system of linear equations, as the first order conditions of the Lagrangian, although initially scary-looking, simplify down to a constant $x/y$ ratio.

All this to say, when you are working with an isoelastic or Cobb-Douglas function in an exercise or in future modules, remember what to look for. If your derivation of the elasticity doesn't result in the $\alpha$ / $\beta$ parameters, or if your derivation of the Lagrangian FOCs does not produce a linear relationship between the two goods, you can be sure there is a mistake!
